In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from pandas.io.json import json_normalize
import requests
import json
from geopy.geocoders import Nominatim
import folium


**Scrapping the given website** 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' **Using Pandas**

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url)

## First check length of df and then extract through length until you get your data

In [ ]:
tor_scrap=df[0]

toronto_assig=(tor_scrap[tor_scrap['Borough']!="Not assigned"]).reset_index(drop=True)

toronto_assig.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [ ]:
toronto_assig.shape


(103, 3)

Uploaded Geospatial file to get co-ordinates

In [ ]:
from google.colab import files
up=files.upload()

Saving Geospatial_Coordinates.csv to Geospatial_Coordinates (1).csv


In [ ]:
tor_data=pd.read_csv('Geospatial_Coordinates.csv')
tor_data.head()



,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [ ]:
tor_data.shape

(103, 3)

In [ ]:
tor_data.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

Now we need to merge the data we got from website and from Geospatial csv file

In [ ]:
Toronto_df=toronto_assig
print(Toronto_df.columns)


Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')


In [ ]:
Toronto_df=Toronto_df.join(tor_data.set_index('Postal Code'),on='Postal Code')


Final Dataframe 

In [ ]:
Toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Will work on the Borough which has keyword Toronto_df

Will filter the dataframe accordingly

In [ ]:
Tor_bor=Toronto_df[Toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Tor_bor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


Lets get the co-ordinates of Toronto

In [ ]:
address='Toronto, ON'
glocator=Nominatim(user_agent='foursquare_agent')
location=glocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

print('The co-ordinates of Toronto are {},{}'.format (latitude,longitude))

The co-ordinates of Toronto are 43.6534817,-79.3839347


Below is the Map with Neighbourhood details

In [ ]:
t_map=folium.Map(location=(latitude,longitude),zoom_start=10)


for lat,lng,name in zip(Tor_bor['Latitude'],Tor_bor['Longitude'],Tor_bor['Neighbourhood']):
  name=folium.Popup(name, parse_html=True)
  folium.CircleMarker(
      location=[lat,lng],
      popup=name,
      radius=5,
      color='blue',
      fill=True,
      fill_color='red',
      parse_html=False,
      fill_opacity=0.6).add_to(t_map)

t_map

In [ ]:
Client_id='ITQ1VVHE2SPGFGQ50SRIQXBZA3RHO2MKUPDNO0LH4N4GD2MG'
Client_secret='4Z3YMXVAXWTZYGAZNQA2BT5CVZK4IXLFLMX2LRNLB0NBBMFQ'
version='20180605'

Now need to explore the venues in each Neighbourhood, lets beign with one to get the clear idea as how to locate the required data

Radius of search is assumed as 500 and Limit is 100

In [ ]:
name=(Tor_bor[['Neighbourhood']].iloc[0,:])[0]
lat=(Tor_bor[['Latitude']].iloc[0,:])[0]
lng=(Tor_bor[['Longitude']].iloc[0,:])[0]
print([name,lat,lng])

['Regent Park, Harbourfront', 43.6542599, -79.3606359]


In [ ]:
url=f'http://api.foursquare.com/v2/venues/explore?client_id={Client_id}&client_secret={Client_secret}&ll={lat},{lng}&v={version}&radius={500}&limit={100}'
url

'http://api.foursquare.com/v2/venues/explore?client_id=ITQ1VVHE2SPGFGQ50SRIQXBZA3RHO2MKUPDNO0LH4N4GD2MG&client_secret=4Z3YMXVAXWTZYGAZNQA2BT5CVZK4IXLFLMX2LRNLB0NBBMFQ&ll=43.6542599,-79.3606359&v=20180605&radius=500&limit=100'

In [ ]:
result=requests.get(url).json()['response']['groups'][0]['items']
result[0]['venue']['name']

'Roselle Desserts'

Defining a function which will take neighbour hood and Lat long as input and provide data frame with venues for neighbourhood

In [ ]:
def get_venue(name,lat,lng):
  venue_list=[]
  for n,l,lo in zip(name,lat,lng):
    url=f'http://api.foursquare.com/v2/venues/explore?client_id={Client_id}&client_secret={Client_secret}&ll={l},{lo}&v={version}&radius={500}&limit={100}'
    result=requests.get(url).json()['response']['groups'][0]['items']
    venue_list.append([(
                       n,
                       l,
                       lo,
                       v['venue']['name'],
                       v['venue']['categories'][0]['name'],
                       v['venue']['location']['lat'],
                       v['venue']['location']['lng']) for v in result])
  d=[item for v_list in venue_list for item in v_list]
  df1=pd.DataFrame(d)
  df1.columns=['Neighbourhood','Neighbour_Latitude','Neighbour_Longitude','Venue_Name','Venue_category','Venue_Latitude','Venue_Longitude']
  return df1

In [ ]:
nearby_venue=get_venue(Tor_bor['Neighbourhood'],Tor_bor['Latitude'],Tor_bor['Longitude'])


In [ ]:
nearby_venue.head()

,Neighbourhood,Neighbour_Latitude,Neighbour_Longitude,Venue_Name,Venue_category,Venue_Latitude,Venue_Longitude
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,Spa,43.654735,-79.359874
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,Restaurant,43.656369,-79.356980


In [ ]:
print("Unique categories {}" .format (len(nearby_venue['Venue_category'])))

Unique categories 1647


Converting Venue categories into numberic values by onehot so that we can apply clustering

In [ ]:
venue_onehot=pd.get_dummies(nearby_venue['Venue_category'])
venue_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
venue_onehot['Neighbourhood']=nearby_venue['Neighbourhood']
venue_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

Neighbourhood column is at the end, will move it to first place

In [ ]:
venue_col_sort=[venue_onehot.columns[-1]]+list(venue_onehot.columns[0:-1])


In [ ]:
venue_onehot=venue_onehot[venue_col_sort]
venue_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0

Groupping by Neighbourhood

In [ ]:
venue_grouped=venue_onehot.groupby('Neighbourhood').mean().reset_index()
venue_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,0.017544,0.000000,0.0,0.0,0.000000,0.017544,0.0,0.017544,0.035088,0.0,0.000000,0.0,0.017544,0.017544,0.0,0.035088,0.0,0.0,0.017544,0.0,0.0000,0.0,0.0000,0.0,0.017544,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.035088,0.0,0.0,0.0,0.0,0.035088,0.000000,0.0,0.0,0.00,0.0,0.035088,0.105263,0.0,0.0,0.0,0.0,0.0,0.017544,0.000000,0.017544,0.00,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.000000,0.0,0.017544,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017544,0.0,0.0,0.0,0.000000,0.035088,0.000000,0.0,0.0,0.017544,0.0,0.0,0.0,0.0,0.017544,0.017544,0.0,0.0,0.0,0.000000,0.0,0.000000

Preparing Data Frame having top 10 venues for each Neighbourhood


In [ ]:
  top_venues=10
  ind=['st','nd','rd']
  col=['Neighbourhood']
  for i in np.arange(top_venues):
    try:
       col.append(f"{i+1}{ind[i]} common_venue")
    except:
      col.append(f"{i+1}th common_venue")
  Neighbourhood_sorted=pd.DataFrame(columns=col)
  Neighbourhood_sorted['Neighbourhood']=venue_grouped['Neighbourhood']

In [ ]:
Neighbourhood_sorted.head()

,Neighbourhood,1st common_venue,2nd common_venue,3rd common_venue,4th common_venue,5th common_venue,6th common_venue,7th common_venue,8th common_venue,9th common_venue,10th common_venue
0,Berczy Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Brockton, Parkdale Village, Exhibition Place",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Business reply mail Processing Centre, South C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"CN Tower, King and Spadina, Railway Lands, Har...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Central Bay Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Listing top 10 common venues

In [ ]:
def extract_values(data,top):
  data_sort=data.sort_values(ascending=False)
  return data_sort.index.values[0:top]


In [ ]:
for i in np.arange(venue_grouped.shape[0]):
  Neighbourhood_sorted.iloc[i,1:]=extract_values(venue_grouped.iloc[i,:].iloc[1:],top_venues)

In [ ]:
Neighbourhood_sorted.head()

,Neighbourhood,1st common_venue,2nd common_venue,3rd common_venue,4th common_venue,5th common_venue,6th common_venue,7th common_venue,8th common_venue,9th common_venue,10th common_venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
4,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Miscellaneous Shop


In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Applying K-means cluster Algorithim

In [ ]:
cluster=5
cluster_data=venue_grouped.drop(columns='Neighbourhood',axis=1)
kmeans=KMeans(n_clusters=cluster,init='k-means++',random_state=0).fit(cluster_data)
#checking the labels
print((kmeans.labels_))


[2 2 0 2 2 0 2 2 0 2 0 2 1 2 2 0 0 2 0 2 1 2 2 2 2 2 1 4 2 2 0 2 2 2 0 3 2
 2 0]


In [ ]:
Neighbourhood_sorted.insert(0,'Cluster',kmeans.labels_)
Neighbourhood_sorted.head()

,Cluster,Neighbourhood,1st common_venue,2nd common_venue,3rd common_venue,4th common_venue,5th common_venue,6th common_venue,7th common_venue,8th common_venue,9th common_venue,10th common_venue
0,2,Berczy Park,Coffee Shop,Restaurant,Bakery,Cocktail Bar,Beer Bar,Farmers Market,Cheese Shop,Seafood Restaurant,Café,Eastern European Restaurant
1,2,"Brockton, Parkdale Village, Exhibition Place",Café,Yoga Studio,Bakery,Breakfast Spot,Coffee Shop,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue
2,0,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa
3,2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Harbor / Marina,Bar,Rental Car Location
4,2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Miscellaneous Shop


Merging Cluster with original data 

In [269]:
Tortonto_merged=Tor_bor
Tortonto_merged=Tortonto_merged.join(Neighbourhood_sorted.set_index('Neighbourhood'),on='Neighbourhood')
Tortonto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st common_venue,2nd common_venue,3rd common_venue,4th common_venue,5th common_venue,6th common_venue,7th common_venue,8th common_venue,9th common_venue,10th common_venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Café,Theater,Breakfast Spot,Health Food Store,Historic Site,Hotel
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,College Auditorium,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Electronics Store,Pizza Place,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Breakfast Spot,Park,Moroccan Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Neighborhood,Health Food Store,Pub,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio


In [ ]:
final_map=folium.Map(location=(latitude,longitude),zoom_start=11)


for lat,lng,name,clu in zip(Tortonto_merged['Latitude'],Tortonto_merged['Longitude'],Tortonto_merged['Neighbourhood'],Tortonto_merged['Cluster']):
  name=folium.Popup(str(name) + ' Cluster ' + str(clu), parse_html=True)
  folium.CircleMarker(
      location=[lat,lng],
      popup=name,
      radius=5,
      color='blue',
      fill=True,
      fill_color='red',
      parse_html=False,
      fill_opacity=0.6).add_to(final_map)

final_map

Examine clusters

In [270]:
Tortonto_merged[Tortonto_merged['Cluster'] == 0]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st common_venue,2nd common_venue,3rd common_venue,4th common_venue,5th common_venue,6th common_venue,7th common_venue,8th common_venue,9th common_venue,10th common_venue
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Nightclub,Italian Restaurant,Diner,Restaurant,Candy Store,Baby Store,Coffee Shop
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Grocery Store,Bar,Supermarket,Middle Eastern Restaurant,Café,Music Venue,Pizza Place,Brewery
15,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Brewery,Movie Theater,Ice Cream Shop,Sushi Restaurant,Fast Food Restaurant,Restaurant,Pub,Italian Restaurant,Board Shop
18,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
22,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,0,Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Furniture / Home Store,Fast Food Restaurant,Bookstore,Cajun / Creole Restaurant,Italian Restaurant,Speakeasy
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,Pizza Place,Pub,Donut Shop,BBQ Joint,Indian Restaurant,History Museum,Pharmacy
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym,Toy / Game Store,Café,Pharmacy
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Restaurant,Bar,Bookstore,Sandwich Place,Japanese Restaurant,Bakery,Chinese Restaurant,Dessert Shop,Pub
35,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Coffee Shop,Pizza Place,Italian Restaurant,Park,Pharmacy,Bakery,Restaurant,Café,Pub,General Entertainment
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0,Light Rail Station,Yoga Studio,Garden Center,Skate Park,Restaurant,Recording Studio,Pizza Place,Park,Garden,Spa


In [ ]:
Tortonto_merged[Tortonto_merged['Cluster'] == 1]

In [ ]:
Tortonto_merged[Tortonto_merged['Cluster'] == 2]

In [ ]:
Tortonto_merged[Tortonto_merged['Cluster'] == 3]

In [ ]:
Tortonto_merged[Tortonto_merged['Cluster'] == 4]